# SimplicialOperator and SimplicialBioperator classes

## Abstract

We present a Python implementation of a class whose instances model simplicial operators. This class supports composition of operators and their action on computable simplicial sets. We also implement a class of pairs of simplicial operators acting on the tensor product of normalized chains.

## Simplicial sets and  the SimplicialOperator class 

A **[simplicial set](https://en.wikipedia.org/wiki/Simplicial_set)** $X$ is a collection of set $\{X_n\}_{n \geq 0}$ together with **degeneracy maps** and **face maps**
\begin{equation*}
s_i : X_n \to X_{n+1} \qquad d_i : X_n \to X_{n-1}
\end{equation*}
for $i = 0, \dots, n$ satisfying the **simplicial identities**:

\begin{align*}
d_i d_j &= d_{j-1} d_i  &\text{ if } i &< j.  \\
d_i s_j &= s_{j-1}d_i    &\text{ if } i &< j. \\
d_i s_j &= \text{id}  &\text{ if } i &= j \text{ or } i = j. \\
d_i s_j &= s_j d_{i-1}   &\text{ if } i &> j. \\
s_i s_j &= s_{j+1} s_i   &\text{ if } i &≤ j.
\end{align*}

We call an element $\sigma \in X_n$ a **simplex of dimension $n$** or **$n$-simplex**. We say that $\sigma$ is a **face** of a simplex $\tau$ if it is the image of $\tau$ via a composition of face maps. We say that $\sigma$ is **degenerate** if it is the image via a degeneracy map of any simplex. 

A **simplicial map** $F : X \to Y$ is a collection of functions $F_n : X_n \to Y_n$ commuting with degeneracy and face maps. 

The set of **simplicial operators** contains the equivalence classes of formal concatenations of symbols $s_i$ and $d_j$ modulo the simplicial identities. A simplicial operator can be uniquely written in the **canonical form**:

\begin{equation*}
s_{u_1} \cdots s_{u_p} d_{v_1} \cdots d_{v_q}
\end{equation*}

with $u_1 > \cdots > u_p$ and $v_1 < \cdots < v_q$.

A simplicial set is said to be **computable** if for any $n$-tuple of $0$-dimensional simplices there exists at most one simplex having it as its ordered collection of $0$-dimensional faces. Examples include: ordered simplicial complexes, directed simplicial complexes, and the nerve of categories.

If $[v_0, \dots, v_n]$ represents an $n$-simplex in a computable simplicial set. The action of any simplicial operator on it is defined by

\begin{equation}
d_i [v_0, \dots, v_n] = [v_0, \dots, \widehat{v}_i, \dots, v_n]
\end{equation}

\begin{equation}
s_i [v_0, \dots, v_n] = [v_0, \dots, v_i, v_i, \dots, v_n].
\end{equation}


In [ ]:
from simplicial_operators import SimplicialOperator

### Example

We will model the operator 
$$op = s_0s_1d_1d_0$$ 
whose action on the simplex $[0,1,2,3]$ is $[1, 1, 3, 3]$.

Its canonical representative is 
$$s_2s_0d_0d_2$$
and we have $op\ op = op$.

In [ ]:
op = SimplicialOperator([0,1],[1,0])

print(f'The canonical representative {op} of our operator is stored.\n')

print(f'Its action on [0,1,2,3] is {op(range(4))}\n')

print(f'We verify this operator is idempotent by composing it with itself and obtaining {op.compose(op)}.')

## Chains, tensor products and simplicial bioperators

Let us consider the **normalized chains with $\mathbb F_2$-coefficients** on a simplicial set 

\begin{equation}
N_n(X) = \frac{\mathbb F_2 \{ X_n \}}{\mathbb F_2 \{ s(X_{n-1}) \}} \ \qquad
\partial_n = \sum_{i=0}^{n} d_{i}
\end{equation}
where $s(X_{n-1}) = \bigcup_{i=0}^{n-1} s_i(X_{n-1})$. 

A **normalized linear simplicial operator** is a formal sum of simplicial operators acting linearly on normalized chains. 

\begin{equation}
\sum d_{u_1} \cdots d_{u_p.}
\end{equation}

In [ ]:
from itertools import product

class LinearSimpOp(set):
    
    def __init__(self, operators, normalized=True):
        if normalized:
            super().__init__({op for op in operators if not op.is_degenerate})
        if not normalized:
            super().__init__(operators)
    
    def __act__(self, simplex):
        pass
    
    def __add__(self, other):
        return LinearSimpOp(self^other)
    
    def __repr__(self):
        return super().__repr__()
        
    def __str__(self):
        return ''.join(f' + {i}' for i in self)[3:]

The **tensor product** of two chain complexes is $C$ and $C'$ is defined by

\begin{equation}
(C \otimes C')_n = \bigoplus_{i+j=n} C_i \otimes C'_j\ \qquad
\partial = \partial \otimes \mathrm{id} + \mathrm{id} \otimes \partial
\end{equation}

A **simplicial bioperator** is the tensor product of two linear simplicial operators acting on $N_*(X) \otimes N_*(Y)$ bilinearly. 

We adapt the SimplicialOperator class to model simplicial bioperators of the form

\begin{equation*}
s_{u_1} \cdots s_{u_p} d_{v_1} \cdots d_{v_q}
\otimes
s_{r_1} \cdots s_{r_l} d_{s_1} \cdots d_{s_t.}
\end{equation*}

In [ ]:
class SimplicialBioperator:
    '''
    Models a simplicial bioperator of form: 

    s ... s d ... d (x) s ... s d ... d
    
    represented in the canonical form
    
    s > ... > s d < ... < d (x) s > ... > s d < ... < d
    '''
    
    def __init__(self, deg_maps1=[], face_maps1=[], deg_maps2=[], face_maps2=[]):
    
        self.op1 = SimplicialOperator(deg_maps1, face_maps1)
        self.op2 = SimplicialOperator(deg_maps2, face_maps2)
    
    def __repr__(self):
        
        s1, d1 = self.op1.deg_maps, self.op1.face_maps
        s2, d2 = self.op2.deg_maps, self.op2.face_maps
    
        return (f'Bioperator(deg_maps1={s1}, face_maps1={d1}, '
                         + f'deg_maps2={s2}, face_maps2={d2}')
    
    def __str__(self):

        return(str(self.op1) + ' x ' + str(self.op2))
    
    def __call__(self, spx1, spx2):
        
        return self.op1(spx1), self.op2(spx2)
    
    def bidegree(self):
        '''returns the bidegree of the operator as a pair of int'''
    
        return self.op1.degree(), self.op2.degree()
    
    def is_degenerate(self):
        '''returns True if the operator is degenerate and False if not'''
        
        s1 = set(self.op1.deg_maps)
        s2 = set(self.op2.deg_maps)
        
        return bool(s1.intersection(s2))
    
    def compose(self, other):
        '''returns the operator self other'''
        
        op1 = self.op1.compose(other.op1)
        op2 = self.op2.compose(other.op2)
        
        return SimplicialBioperator(op1.deg_maps, op1.face_maps, 
                                    op2.deg_maps, op2.face_maps)

### Example

We will model the operator 
$$biop = s_0s_1d_1d_0 \otimes s_1 d_0 d_1$$ 
whose action on the simplex $[0,1,2,3]$ is $[1, 1, 3, 3]$.

Its canonical representative is 
$$s_2s_0d_0d_2 x s_1d_0d_1$$
and we have $op\ op = op$.

In [ ]:
biop = SimplicialBioperator([0,1],[1,0],[1],[0,1])

print(f'The canonical representative {biop} of our operator is stored.\n')

action = biop(range(4),range(5))
print(f'Its action on [0,1,2,3] x [0,1,2,3,4] is {action[0]} x {action[1]}\n')

print(f'The composition with itself is {biop.compose(biop)}.')

biop